This code generates the C wrapper to run the parametric simplex method in Python using the source files in the fastclime R package.

In [1]:
from __future__ import division
import os
import sys
import glob

In [2]:
%%file parametric.h

int ratio_test(double *dy, int *idy,int ndy, double *y, double mu);
void parametric(double *SigmaInput, int *m1, double *mu_input, double *lambdamin, int *nlambda, int *maxnlambda, double *iicov);
void solver2(int m,int n,int nz,int *ia, int *ka, double *a,double *b, double *c);

Overwriting parametric.h


In [3]:
%%file _parametric.pyx

import numpy as np
cimport numpy as np

np.import_array()

# cdefine the signature of the c function
cdef extern from "parametric.h":
    void parametric(double *SigmaInput, int *m1, double *mu_input, 
                    double *lambdamin, int *nlambda, int *maxnlambda, double *iicov);
    
def mainfunc(np.ndarray[double, ndim=2, mode="c"] SigmaInput not None, 
             double lambdamin, 
             int nlambda):   

    #Dimensions    
    cdef int m1 = SigmaInput.shape[1]
 
    #Define output
    cdef int maxnlambda = 0
    cdef np.ndarray mu_input = np.zeros((m1,nlambda), dtype = np.float64, order='C')
    cdef np.ndarray iicov = np.zeros((m1*m1,nlambda), dtype = np.float64, order='C')

    #Call external C function
    parametric(<double*> np.PyArray_DATA(SigmaInput),
               &m1,
               <double*> np.PyArray_DATA(mu_input),
               <double*> &lambdamin,
               &nlambda,
               &maxnlambda,
               <double*> np.PyArray_DATA(iicov)
              )
    
    return (SigmaInput,mu_input,maxnlambda,iicov)


Overwriting _parametric.pyx


In [4]:
%%file setup.py

from distutils.core import setup, Extension
import numpy as np
import numpy.distutils.misc_util
from Cython.Distutils import build_ext

setup(
    name = 'parametric',
    version = '1.0',
    description = 'C wrapper for parametric function created by Haotian Pang et. al',
    author = 'John Pura',
    author_email = 'john.pura@duke.edu',
    cmdclass={'build_ext': build_ext},
    ext_modules=[Extension("parametric",
                           sources=["_parametric.pyx", "parametric.c",
                                    "lu.c","tree.c","linalg.c",
                                    "heap.c"],
                 include_dirs=[np.get_include(), ])]
)


Overwriting setup.py


In [5]:
!python setup.py build_ext --inplace

running build_ext
cythoning _parametric.pyx to _parametric.c
building 'parametric' extension
C compiler: gcc -pthread -fno-strict-aliasing -g -O2 -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC

compile options: '-I/home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include -I/home/bitnami/anaconda/include/python2.7 -c'
gcc: parametric.c
gcc: heap.c
gcc: linalg.c
gcc: tree.c
gcc: lu.c
gcc: _parametric.c
In file included from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1804:0,
                 from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:17,
                 from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from _parametric.c:232:
/home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining